In [1]:
sc.master

'local[*]'

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Row
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank
from pyspark.sql.functions import row_number

In [4]:
UserAcessLog = ["2019-05-29,aa,3","2019-05-29,bb,12",
                         "2019-05-29,aa,65","2019-05-29,cc,",
                         "2019-05-30,aa,56","2019-05-30,ee,45",
                         "2019-05-30,aa,45","2019-05-30,aa,30"
                        ]
LogStrRDD = sc.parallelize(UserAcessLog,1).filter(lambda line: True if (len(line.split(',')[2])>0) else False)

LogStrRDD = LogStrRDD.map(lambda line  :\
                     Row(line.split(",")[0], 
                         line.split(",")[1],
                        int(line.split(",")[2])))
# 
schema = StructType([StructField("date",StringType()),
                     StructField("userid",StringType()),
                     StructField("click",IntegerType())])


In [56]:
#row里如果有int型，直接take在python会在报错row里如果有int型，
#直接take在python会在报错
#LogStrRDD.take(20) 

In [5]:
LogDF = spark.createDataFrame(LogStrRDD,schema)
LogDF.show(10)

+----------+------+-----+
|      date|userid|click|
+----------+------+-----+
|2019-05-29|    aa|    3|
|2019-05-29|    bb|   12|
|2019-05-29|    aa|   65|
|2019-05-30|    aa|   56|
|2019-05-30|    ee|   45|
|2019-05-30|    aa|   45|
|2019-05-30|    aa|   30|
+----------+------+-----+



In [61]:
# Python版本agg里 只能用alias别名，as报错
LogDF.groupBy("date").agg(
    countDistinct("userid").alias("ss"),
    sum("click").alias("click_sum")).show()

+----------+---+---------+
|      date| ss|click_sum|
+----------+---+---------+
|2019-05-29|  2|       80|
|2019-05-30|  2|      176|
+----------+---+---------+



# 开窗函数：分组求topn

In [14]:
LogDF.printSchema

<bound method DataFrame.printSchema of DataFrame[date: string, userid: string, click: int]>

In [24]:
LogDF[["date","userid","click",(LogDF.click+10).alias("new")]].show()

+----------+------+-----+---+
|      date|userid|click|new|
+----------+------+-----+---+
|2019-05-29|    aa|    3| 13|
|2019-05-29|    bb|   12| 22|
|2019-05-29|    aa|   65| 75|
|2019-05-30|    aa|   56| 66|
|2019-05-30|    ee|   45| 55|
|2019-05-30|    aa|   45| 55|
|2019-05-30|    aa|   30| 40|
+----------+------+-----+---+



In [20]:
LogDF.select("date","userid","click",
             dense_rank().over(
                Window.partitionBy("date").orderBy("click")).alias("rank")
            ).show()

+----------+------+-----+----+
|      date|userid|click|rank|
+----------+------+-----+----+
|2019-05-29|    aa|    3|   1|
|2019-05-29|    bb|   12|   2|
|2019-05-29|    aa|   65|   3|
|2019-05-30|    aa|   30|   1|
|2019-05-30|    ee|   45|   2|
|2019-05-30|    aa|   45|   2|
|2019-05-30|    aa|   56|   3|
+----------+------+-----+----+

